In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle 
import sys
import seaborn as sb
import math
from matplotlib.ticker import LinearLocator

In [2]:
sys.path.append(r'C:\Users\Joar\Documents\1_Education\NTNU\OneDrive - NTNU\Thesis\Modelling\FD Model')

In [3]:
from FloaterParameters import FloaterParameters
from Environment import Environment
from Buoy import Buoy
from Mass import Mass
from Density import Density
from Area import Area
from GeneralisedCoordinateSystem import GeneralisedCoordinateSystem
from SystemMatrices import MatrixCalculation
from ComputeHydroCoefficients import CompHydroCoefficient
from plot_hydroD_results import plot_hydroD_results
from ReadWadamLis import ReadWadamLis
from CheckInterpolation import InterpolateParameters
from kSolve import ksolve

Unable to import mpi4py. Parallel processing unavailable.
Unable to import petsc4py. Parallel processing unavailable.
Unable to import petsc4py. Parallel processing unavailable.


In [4]:
plt.rcParams['axes.grid'] = True
plt.rcParams["figure.figsize"] = (20, 10)
plt.rcParams.update({'font.size': 20})

In [5]:
# To be used if running simulations
interpolate = 0
pull_results = 0
write_one = 0
write = 0
run = 0
parameters = np.array([[90,90]])
column_diameter = [12]

In [6]:
mufp = FloaterParameters(parameters[0,0],parameters[0,1], column_diameter[0])
env = Environment()
rho = Density()
csa = Area(mufp)
buoy = Buoy(mufp, csa, rho)
mass = Mass(mufp, csa, buoy, rho)
coord = GeneralisedCoordinateSystem(mufp, csa, mass, rho, buoy, env)

In [7]:
path = r'C:\Users\Joar\Documents\1_Education\NTNU\WadamRun1\WADAM1.LIS'
hydroD_results = open(path, 'r').readlines()

In [8]:
found = 0
ii = 0

In [9]:
ii = 1
found = 0
while found == 0 and ii < len(hydroD_results):
    k = hydroD_results[ii].find(
        '    THE OUTPUT IS NON-DIMENSIONALIZED USING -\n')
    ii += 1
    if k != -1:
        found = 1
        RO = float(hydroD_results[ii + 7].split()[2])
        G = float(hydroD_results[ii + 8].split()[2])
        VOL = float(hydroD_results[ii + 9].split()[2])
        L = float(hydroD_results[ii + 10].split()[2])
        WA = float(hydroD_results[ii + 11].split()[2])

In [10]:
found = 0
ii = 0

In [11]:
while found == 0:
    k = hydroD_results[ii].find('2.8 ENVIRONMENTAL DATA:\n')
    ii += 1
    if k != -1:
        found = 1
        waterdepth = float(hydroD_results[ii + 2].split()[3])
        numwavelengths = int(hydroD_results[ii + 3].split()[5])
        numheadangles = int(hydroD_results[ii + 4].split()[5])
        wavedata = np.zeros(shape=(numwavelengths, numheadangles, 5))

        for jj in np.linspace(1, numheadangles, numheadangles).astype(int) - 1:
            for kk in np.linspace(1, numwavelengths, numwavelengths).astype(int) - 1:
                wavedata[kk, jj, :] = np.float_(hydroD_results[ii + kk + 12].split())

ii = 1
found = 0


In [12]:
wave_disc = np.zeros(shape=(int(numwavelengths), 5))

while found == 0:
    k = hydroD_results[ii].find('     WAVE DESCRIPTION:\n')
    ii += 1
    if k != -1:
        found = 1
        for jj in np.linspace(0, int(numwavelengths) - 1, int(numwavelengths)).astype(int):
            wave_disc[jj, :] = np.float_(hydroD_results[ii + 4 + jj].split())

In [13]:
ii = 1
found = 0

while found == 0:
    k = hydroD_results[ii].find(' 2.3.1 DATA SPECIFYING THE PANEL MODEL:\n')
    ii += 1
    if k != -1:
        found = 1
        num_wet_panels = int(hydroD_results[ii+39].split()[6])
        num_panels = int(hydroD_results[ii+40].split()[10])
        num_dry_panels = int(hydroD_results[ii+41].split()[8])

In [14]:
panel_pressure = np.zeros(shape=(numheadangles, numwavelengths, 2* num_wet_panels+1, 21))

In [15]:
ii = 1
found = 0

In [16]:
found = 0
kk = 0
ii = 0
panel_area = np.zeros(shape=(5,num_wet_panels))
while found == 0:
    k = hydroD_results[ii].find('   PI   PANO IVER X         Y         Z          NX      NY      NZ     XC        YC        ZC        AREA      DIAG\n')
    ii += 1
    if k != -1:
        ii += 3
        for jj in np.arange(0,15,1):
            if hydroD_results[ii+5*jj] == '1\n':
                break
            panel_area[0,kk] = hydroD_results[ii+5*jj].split()[0]
            panel_area[1,kk] = hydroD_results[ii+5*jj].split()[-2]
            panel_area[2, kk] = float(''.join(list(hydroD_results[ii+5*jj])[48:70]).split()[0])
            panel_area[3, kk] = float(''.join(list(hydroD_results[ii+5*jj])[48:70]).split()[1])
            panel_area[4, kk] = float(''.join(list(hydroD_results[ii+5*jj])[48:70]).split()[2])
            kk += 1
            if int(hydroD_results[ii+5*jj].split()[0]) == num_wet_panels:
                found = 1
                break

In [22]:
found = 0
kk = 0
ii = 0

In [23]:
for bb in np.arange(0,numwavelengths,1):
    for aa in np.arange(0,numheadangles,1):
        kk = 0
        found = 0
        while found == 0:
            k = hydroD_results[ii].find('    BASIC PART                                                                                                                     \n')
            ii += 1
            if k != -1:
                found = 1
                for jj in np.arange(0,41,1):
                    if hydroD_results[ii+5+jj] == '1\n':
                        break
                    panel_pressure[aa,bb,kk,0:11] = hydroD_results[ii+5+jj].split()
                    panel_pressure[aa,bb,kk,11] = panel_area[1,int(panel_pressure[aa,bb,kk,0])-1]
                    # [real, imag, abs]* area * normal direction
                    panel_pressure[aa,bb,kk,12:15] = panel_pressure[aa,bb,kk,7:10]*panel_pressure[aa,bb,kk,11]*panel_area[2, int(panel_pressure[aa,bb,kk,0])-1]
                    panel_pressure[aa,bb,kk,15:18] = panel_pressure[aa,bb,kk,7:10]*panel_pressure[aa,bb,kk,11]*panel_area[3, int(panel_pressure[aa,bb,kk,0])-1]
                    panel_pressure[aa,bb,kk,18:21] = panel_pressure[aa,bb,kk,7:10]*panel_pressure[aa,bb,kk,11]*panel_area[4, int(panel_pressure[aa,bb,kk,0])-1]
                    kk += 1

        found = 0     

        while found == 0:
            k = hydroD_results[ii].find('    BASIC PART  CONT.                                                                                                              \n')
            ii += 1
            if k != -1:
                for jj in np.arange(0,41,1):
                    if hydroD_results[ii+5+jj] == '1\n':
                        break
                    panel_pressure[aa,bb,kk,0:11] = hydroD_results[ii+5+jj].split()
                    panel_pressure[aa,bb,kk,11] = panel_area[1,int(panel_pressure[aa,bb,kk,0])-1]
                    # [real, imag, abs]* area * normal direction
                    panel_pressure[aa,bb,kk,12:15] = panel_pressure[aa,bb,kk,7:10]*panel_pressure[aa,bb,kk,11]*panel_area[2, int(panel_pressure[aa,bb,kk,0])-1]
                    panel_pressure[aa,bb,kk,15:18] = panel_pressure[aa,bb,kk,7:10]*panel_pressure[aa,bb,kk,11]*panel_area[3, int(panel_pressure[aa,bb,kk,0])-1]
                    panel_pressure[aa,bb,kk,18:21] = panel_pressure[aa,bb,kk,7:10]*panel_pressure[aa,bb,kk,11]*panel_area[4, int(panel_pressure[aa,bb,kk,0])-1]
                    kk += 1
                    if int(hydroD_results[ii+5+jj].split()[0]) == num_wet_panels:
                        found = 1
                        break
        
        found = 0

        while found == 0:
            k = hydroD_results[ii].find('    1. REFLECTION                                                                                                                  \n')
            ii += 1
            if k != -1:
                for jj in np.arange(0,41,1):
                    if hydroD_results[ii+5+jj] == '1\n':
                        found = 1
                        break
                    panel_pressure[aa,bb,kk,0:11] = hydroD_results[ii+5+jj].split()
                    panel_pressure[aa,bb,kk,11] = panel_area[1,int(panel_pressure[aa,bb,kk,0])-1]
                    # [real, imag, abs]* area * normal direction
                    panel_pressure[aa,bb,kk,12:15] = panel_pressure[aa,bb,kk,7:10]*panel_pressure[aa,bb,kk,11]*panel_area[2, int(panel_pressure[aa,bb,kk,0])-1]
                    panel_pressure[aa,bb,kk,15:18] = panel_pressure[aa,bb,kk,7:10]*panel_pressure[aa,bb,kk,11]*-1*panel_area[3, int(panel_pressure[aa,bb,kk,0])-1]
                    panel_pressure[aa,bb,kk,18:21] = panel_pressure[aa,bb,kk,7:10]*panel_pressure[aa,bb,kk,11]*panel_area[4, int(panel_pressure[aa,bb,kk,0])-1]
                    kk += 1
        found = 0

        while found == 0:
            k = hydroD_results[ii].find('    1. REFLECTION  CONT.                                                                                                           \n')
            ii += 1
            if k != -1:
                for jj in np.arange(0,41,1):
                    if hydroD_results[ii+5+jj] == '1\n':
                        break
                    panel_pressure[aa,bb,kk,0:11] = hydroD_results[ii+5+jj].split()
                    panel_pressure[aa,bb,kk,11] = panel_area[1,int(panel_pressure[aa,bb,kk,0])-1]
                    # [real, imag, abs]* area * normal direction
                    panel_pressure[aa,bb,kk,12:15] = panel_pressure[aa,bb,kk,7:10]*panel_pressure[aa,bb,kk,11]*panel_area[2, int(panel_pressure[aa,bb,kk,0])-1]
                    panel_pressure[aa,bb,kk,15:18] = panel_pressure[aa,bb,kk,7:10]*panel_pressure[aa,bb,kk,11]*-1*panel_area[3, int(panel_pressure[aa,bb,kk,0])-1]
                    panel_pressure[aa,bb,kk,18:21] = panel_pressure[aa,bb,kk,7:10]*panel_pressure[aa,bb,kk,11]*panel_area[4, int(panel_pressure[aa,bb,kk,0])-1]
                    kk += 1
                    if int(hydroD_results[ii+5+jj].split()[0]) == num_wet_panels:
                        found = 1
                        break

In [24]:
panel_pressure[:,:,:,7:10] *= RO*G*WA
panel_pressure[:,:,:,12:21] *= RO*G*WA

In [26]:
hydroD_results[34138]

'    EXCITING FORCES AND MOMENTS FROM INTEGRATION OF PRESSURE                                                                       \n'

In [ ]:
plt.figure()
plt.plot(panel_pressure[0,50,:,12],label='50')
plt.plot(panel_pressure[0,51,:,12], label='51')
plt.plot(panel_pressure[0,52,:,12], label='52')
plt.legend()

In [ ]:
plt.figure()
plt.plot(panel_pressure[0,50,:,15], label='50')
plt.plot(panel_pressure[0,51,:,15], label='51')
plt.plot(panel_pressure[0,52,:,15], label='52')
plt.legend()

In [ ]:
plt.figure()
plt.plot(panel_pressure[0,50,:,18], label='50')
plt.plot(panel_pressure[0,51,:,18], label='51')
plt.plot(panel_pressure[0,52,:,18], label='52')
plt.legend()

####        PI   PANO SET IND   XC         YC         ZC          REAL PART   IMAG.PART    ABS.VALUE   PHASE ANGLE(DEG)

In [ ]:
front_column = panel_pressure[:,:,np.where(panel_pressure[0,0,:,4]<-5)[0],:].copy()
left_column = panel_pressure[:,:,np.where(panel_pressure[0,0,:,5]<-15)[0],:].copy()
right_column = panel_pressure[:,:,np.where(panel_pressure[0,0,:,5]>15)[0],:].copy()

#### Calculating Net Force as a function of frequency
- Phase angle important but would probably already be accounted for by wavelength
- A + jB
- Lets take the absolute

Lets sum the pressure * area of each of the columns

    PRESSURE DISTRIBUTION     INCLUDE PRESSURES FROM BOTH THE RADIATION- AND DIFFRACTION VELOCITY POTENTIALS.
                              NO FLUCTUATING HYDROSTATIC PRESSURES DUE TO MOTION ARE INCLUDED.

In [ ]:
front_column_force = np.zeros(shape=(2,84,9))
left_column_force = np.zeros(shape=(2,84,9))
right_column_force = np.zeros(shape=(2,84,9))

In [ ]:
for ii in np.arange(0, numheadangles, 1).astype(int):
    for jj in np.arange(0, numwavelengths, 1).astype(int):
        front_column_force[ii, jj, :] = np.sum(front_column[ii, jj, :, 12:21], axis=0)

for ii in np.arange(0, numheadangles, 1).astype(int):
    for jj in np.arange(0, numwavelengths, 1).astype(int):
        left_column_force[ii, jj, :] = np.sum(left_column[ii, jj, :, 12:21], axis=0)

for ii in np.arange(0, numheadangles, 1).astype(int):
    for jj in np.arange(0, numwavelengths, 1).astype(int):
        right_column_force[ii, jj, :] = np.sum(right_column[ii, jj, :, 12:21], axis=0)

In [ ]:
pd.DataFrame(front_column[0,0,:,12:21])

In [ ]:
pd.DataFrame(front_column_force[0,:,:])

In [ ]:
plt.figure()
plt.plot(wave_disc[:, 4], front_column_force[0,:,0], label='real - x')
plt.plot(wave_disc[:, 4], front_column_force[0,:,1], '-*', label='imag - x')
plt.plot(wave_disc[:, 4], front_column_force[0,:,2], label='abs - x')
#for xc in omega_x:
#    plt.axvline(x=xc, linestyle='--', linewidth=2, color='c')
for xc in omega_y:
    plt.axvline(x=xc, linestyle='--', linewidth=2, color='m')
#for xc in omega_xy:
#    plt.axvline(x=xc, linestyle='--', linewidth=2, color='y')  
plt.ticklabel_format(axis="y", style="sci", scilimits=(0, 0))
plt.xlabel('Frequency [rad/s]')
plt.ylabel('Force [N]')
plt.ylim([-5e6,1e7])
plt.legend()

In [ ]:
plt.figure()
plt.plot(wave_disc[:, 4], front_column_force[0,:,3],'-*', label='real - y')
plt.plot(wave_disc[:, 4], front_column_force[0,:,4], label='imag - y')
plt.plot(wave_disc[:, 4], front_column_force[0,:,5], label='abs - y')
plt.ticklabel_format(axis="y", style="sci", scilimits=(0, 0))
plt.xlabel('Frequency [rad/s]')
plt.ylabel('Force [N]')
plt.ylim([-5e6,5e6])
plt.legend()

In [ ]:
plt.figure()
plt.plot(wave_disc[:, 4], front_column_force[0,:,6],'-*', label='real - z')
plt.plot(wave_disc[:, 4], front_column_force[0,:,7], label='imag - z')
plt.plot(wave_disc[:, 4], front_column_force[0,:,8], label='abs - z')
plt.ticklabel_format(axis="y", style="sci", scilimits=(0, 0))
plt.xlabel('Frequency [rad/s]')
plt.ylabel('Force [N]')
plt.ylim([-5e6,5e6])
plt.legend()

In [ ]:
plt.figure()
plt.plot(wave_disc[:, 4], front_column_force[0,:,8], label='abs - front')
plt.plot(wave_disc[:, 4], left_column_force[0,:,8], label='abs - left')
plt.plot(wave_disc[:, 4], right_column_force[0,:,8], label='abs - right')
plt.ticklabel_format(axis="y", style="sci", scilimits=(0, 0))
plt.xlabel('Frequency [rad/s]')
plt.ylabel('Force [N]')
plt.ylim([-5e6,1e7])
plt.legend()

In [ ]:
L_d = []
k_d = []
omega_d  = np.zeros(21)
for jj, i in enumerate(np.arange(1., 20., 0.1)):
    L_d.append(mufp.dia_column / i)
    k_d.append(2 * np.pi / L_d[jj])
L_d = np.array(L_d)
k_d = np.array(k_d)
omega_d = np.sqrt(env.g * k_d * np.tanh(k_d * env.h))
omega_d = np.delete(omega_d, np.where(omega_d > 5))

In [ ]:
#for xc in omega_x:
#    plt.axvline(x=xc, linestyle='--', linewidth=2, color='c')
#for xc in omega_y:
#    plt.axvline(x=xc, linestyle='--', linewidth=2, color='m')
#for xc in omega_xy:
#    plt.axvline(x=xc, linestyle='--', linewidth=2, color='y')   

In [ ]:
L_x = np.zeros(21)
k_x = np.zeros(21)

L_y = np.zeros(21)
k_y = np.zeros(21)
L_xy = np.zeros(21)
k_xy = np.zeros(21)

omega_list = np.arange(0.01, 5, 0.01)
k_list = np.zeros(len(omega_list))
L_list = np.zeros(len(omega_list))

phi_x = np.zeros(len(omega_list))
phi_y = np.zeros(len(omega_list))
phi_xy = np.zeros(len(omega_list))


for ii in np.linspace(0, len(omega_list) - 1, len(omega_list)).astype(int):
    [k_list[ii], L_list[ii]] = ksolve(omega_list[ii], 50, 9.81)
    phi_x[ii] = (mufp.x_space / L_list[ii]) - np.floor(mufp.x_space / L_list[ii])
    phi_y[ii] = (mufp.y_space / L_list[ii]) - np.floor(mufp.y_space / L_list[ii])
    phi_xy[ii] = (np.sqrt(mufp.y_space**2 + mufp.x_space**2)  / L_list[ii]) - np.floor(np.sqrt(mufp.y_space**2 + mufp.x_space**2)  / L_list[ii])

phi_x = phi_x * 360 - 180
phi_y = phi_y * 360 - 180

for i in np.linspace(1, 20, 20).astype(int):
    L_x[i] = mufp.x_space / i
    k_x[i] = 2 * np.pi / L_x[i]
omega_x = np.sqrt(env.g * k_x * np.tanh(k_x * env.h))
omega_x = np.delete(omega_x, np.where(omega_x < 0.5))
omega_x = np.delete(omega_x, np.where(omega_x > 5))

for i in np.linspace(1, 20, 20).astype(int):
    L_y[i] = mufp.y_space / i
    k_y[i] = 2 * np.pi / L_y[i]
omega_y = np.sqrt(env.g * k_y * np.tanh(k_y * env.h))
omega_y = np.delete(omega_y, np.where(omega_y < 0.5))
omega_y = np.delete(omega_y, np.where(omega_y > 5))

for i in np.linspace(1, 20, 20).astype(int):
    L_xy[i] = np.sqrt(mufp.y_space**2 + mufp.x_space**2)   / i
    k_xy[i] = 2 * np.pi / L_xy[i]
omega_xy = np.sqrt(env.g * k_xy * np.tanh(k_xy * env.h))
omega_xy = np.delete(omega_xy, np.where(omega_xy < 0.5))
omega_xy = np.delete(omega_xy, np.where(omega_xy > 5))